In [ ]:
# default_exp core

# core

> core stuff.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)

In [ ]:
# export


def ga_to_df(qry: dict, view_id = None) -> pd.DataFrame:
    if not view_id:
        view_id = os.getenv('GOOGLE_ANALYTICS_VIEW_ID')
    qry['viewId'] = view_id
    analytics = build('analyticsreporting', 'v4', credentials=credentials)
    request_list = [qry]
    response = analytics.reports().batchGet(body={'reportRequests': request_list}).execute()
    data = []
    for report in response.get('reports'):
        dimensions = report.get('columnHeader').get('dimensions')
        metrics = [x['name'] for x in report.get('columnHeader').get('metricHeader').get('metricHeaderEntries')]
        for row in report.get('data').get('rows'):
            dimensions_data = row.get('dimensions', [])
            metrics_data = [x['values'] for x in row.get('metrics', [])][0]
            metrics_data = [float(x) for x in metrics_data]
            colnames = dimensions + metrics
            data_row = dimensions_data + metrics_data
            data.append(data_row)
    df = pd.DataFrame(data, columns=colnames)
    return df



In [ ]:
# test

# make a example query
qry = {
    'dateRanges': [{'startDate': '7daysAgo', 'endDate': 'today'}],
    'metrics': [{'expression': 'ga:sessions'}, {'expression': 'ga:pageviews'}],
    'dimensions': [{'name': 'ga:date'}, {'name': 'ga:country'}]
}

# get the df
df = ga_to_df(qry)

# check have realistic looking data
assert df['ga:sessions'].sum() >= 0
assert df['ga:pageviews'].sum() >= 0
assert df['ga:pageviews'].sum() >= df['ga:sessions'].sum()